In [1]:
%load_ext autoreload
%autoreload 2


In [2]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from pathlib import Path
import seaborn as sns

In [3]:
import os
import sys

In [4]:
# chaging directory

In [5]:
root, *_ = !pwd

In [6]:
if root.endswith('notebooks'):
    os.chdir('../')
    root, *_ = !pwd
    sys.path.append('src/')

In [7]:
os.getcwd()

'/home/med/projects/competitions/mlb-kaggle'

In [8]:
from dotenv import load_dotenv
load_dotenv()

True

In [9]:
RAW_PATH = Path(os.environ['RAW_PATH'])

In [10]:
PROCESSED_PATH = Path(os.environ['ROOT_DIR']) / 'processed'

In [11]:
os.listdir(RAW_PATH)

['seasons.csv',
 'teams.csv',
 'example_sample_submission.csv',
 'train.csv',
 'mlb-player-digital-engagement-forecasting.zip',
 'awards.csv',
 'players.csv',
 'example_test.csv',
 'mlb']

In [12]:
os.listdir(PROCESSED_PATH)

['target.csv', 'raw_data.csv']

In [13]:
raw_train_data = pd.read_csv(RAW_PATH / 'train.csv', parse_dates=['date'])

### players stats

In [21]:
from pipeline.core import make_unpack_tmf

# pstats_col = 'playerBoxScores'  
# # check if there is any information
# # normalize dataframe
# unpack_pstats = make_unpack_tmf(pstats_col)
# pstats = unpack_pstats.fit_transform(raw_train_data)

In [140]:

# def ingest_games_stats(pstats: pd.DataFrame, raw_df: pd.DataFrame):
#     games_col = 'games'  
#     # check if there is any information
#     if raw_df.loc[:, games_col].isna().all():
#         return None
    
#     unpack_games = make_unpack_tmf(games_col)
#     games = unpack_games.fit_transform(raw_df)
    
#     teamFeatures = ['Id', 'Wins', 'Losses', 'WinPct', 'Winner', 'Score']

#     add_suffix = lambda suffix: [suffix + f for f in teamFeatures] 

#     homeFeatures = add_suffix('home')
#     awayFeatures = add_suffix('away')
#     playerTeamFeatures = add_suffix('playerTeam')
#     opTeamFeatures = add_suffix('opponentTeam')
    
#     to_drop = ['season', 'gameTimeUTC', 'resumeDate',
#                 'resumedFrom', 'codedGameState', 'detailedGameState',
#                 'gameNumber', 'doubleHeader', 'dayNight', 'scheduledInnings',
#                 'seriesDescription', 'homeName',
#                 'homeAbbrev','homeName',
#                 'awayName', 'awayAbbrev', 'date']
    
#     # drop features
#     games.drop(to_drop, axis=1, inplace=True)
#     # merge with correct gameId
#     pstats = pstats.merge(games, on=['gamePk', 'gameDate'], how='left')
#     # home and aways stats to player and opponent stats
#     pstats.loc[:, playerTeamFeatures] = np.where(pstats[['home']], pstats[homeFeatures], pstats[awayFeatures])
#     pstats.loc[:, opTeamFeatures] = np.where(pstats[['home']], pstats[awayFeatures], pstats[homeFeatures])
#     # drop features
#     pstats.drop(homeFeatures + awayFeatures +
#                 ['homeId', 'awayWinner',
#                  'opponentTeamWinner'], axis=1, inplace=True)
#     # compute the difference between the scores
#     pstats['teamScoreDiff'] = pstats['playerTeamScore'] - pstats['opponentTeamScore'] 
#     return pstats


        

from data.ingest_data import ingest_player_stats

In [141]:
pstats = ingest_player_stats(raw_train_data)

In [130]:
pstats = ingest_games_stats(pstats, raw_train_data)

In [142]:
pstats.head()

,home,teamId,playerId,date,gameType,isTie,gamesInSeries,playerTeamId,playerTeamWins,playerTeamLosses,...,catchersInterferencePitching,sacBuntsPitching,sacFliesPitching,saves,holds,blownSaves,assists,putOuts,errors,chances
0,1,119,605131,87,R,0.0,4.0,119,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1,139,605480,87,R,0.0,4.0,139,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0,143,546318,87,R,0.0,3.0,143,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0,108,527043,87,R,0.0,4.0,108,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0
4,1,118,449181,87,R,0.0,2.0,118,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,2.0


In [135]:
to_drop = ['gamePk',
           'gameTimeUTC',
           'teamName',
           'playerName',
           'jerseyNum',
           'positionCode',
           'positionName',
           'positionType']

In [2]:
pd.api.types.is_float_dtype

NameError: name 'df' is not defined

In [137]:
pstats.drop(to_drop, axis=1)

Index(['home', 'gameDate', 'teamId', 'playerId', 'date', 'battingOrder',
       'gamesPlayedBatting', 'flyOuts', 'groundOuts', 'runsScored', 'doubles',
       'triples', 'homeRuns', 'strikeOuts', 'baseOnBalls', 'intentionalWalks',
       'hits', 'hitByPitch', 'atBats', 'caughtStealing', 'stolenBases',
       'groundIntoDoublePlay', 'groundIntoTriplePlay', 'plateAppearances',
       'totalBases', 'rbi', 'leftOnBase', 'sacBunts', 'sacFlies',
       'catchersInterference', 'pickoffs', 'gamesPlayedPitching',
       'gamesStartedPitching', 'completeGamesPitching', 'shutoutsPitching',
       'winsPitching', 'lossesPitching', 'flyOutsPitching', 'airOutsPitching',
       'groundOutsPitching', 'runsPitching', 'doublesPitching',
       'triplesPitching', 'homeRunsPitching', 'strikeOutsPitching',
       'baseOnBallsPitching', 'intentionalWalksPitching', 'hitsPitching',
       'hitByPitchPitching', 'atBatsPitching', 'caughtStealingPitching',
       'stolenBasesPitching', 'inningsPitched', 'saveOpp